# Assignment 1 - Search

Class: COMP 5600 - Artificial Intelligence

Author: Chris Hinkson

Email: cmh0201@auburn.edu

## Assignment Report

### Design Choices

As the assignment description stated, we would need to compare Breadth First Search, Depth First Search, and A* Search Algorithms across three different test cases. 

For the A* Search Algorithms, the assignment specified that we needed to use at least two different heuristics. I decided to compare three different heuristics for A* Search: Euclidean Distance Heuristic (`h(n) = sqrt((x2 - x1)^2 + (y^2 - y1)^2)`), Manhattan Distance Heuristic (`h(n) = |x2 - x1| + |y2 - y1|`), and Chebyshev Distance Heuristic (`h(n) = MAX(|x2 - x1|, |y2-y1|)`). Assuming that these algorithms will not be performed on any data that allows diagonal jumps (as neither of the three test cases do), all three of these heuristics are admissible, since for any node n the estimated cost h(n) will never be greater than the true cost h*(n) such that h(n) <= h*(n). I chose these three heuristics because they are all admissible, all relatively simple to calculate, and all deal with some form of distance measurement to the end goal.

To compare the algorithms, I decided that execution time and total number of nodes visited on the frontier would be good metrics to analyze. These were suggested as part of the assignment description and are common ways of analyzing program / algorithm performance. Ideally, these will allow for the various algorithms to be compared in such a way that it demonstrates which algorithms perform better than others.

### Implementation Strategy

To compare the algorithms, I wanted the implementation of each to be fair as far as possible so that the algorithm itself is being compared and not secondary tasks. Additionally, to compare the algorithms efficiently, I needed a way to store the metrics and results for each algorithm across test cases. As such, I created three utility classes:

- A DataUtilities Class to gather data from given txt and csv files and prepare it in advance of the algorithm being ran.
- An AlgorithmTimer Class to compute and store algorithm execution times.
- An AlgorithmRecords Class to store algorithm results.

For each algorithm, I created a new cell that would house a global function to run the algorithm on a given dataset. These would allow each algorithm to be run on its own while the end results are still being tracked. Each algorithm cell contains the algorithm along with the code necessary to run the algorithm and track metrics about it.

For BFS and DFS, two lists are used that allow the algorithm to add new nodes to the frontier and keep track of which nodes are visited on the frontier. The way that elements are added to the frontier list implements the strategy of the algorithm:
- In BFS, as new nodes are discovered, they are added to the end of the frontier list. The algorithm will continue to visit all nodes on the current level (which are already all in the frontier) before visiting these newly discovered nodes.
- In DFS, as new nodes are discovered, they are added before any other discovered nodes in the frontier list. The algorithm will continue to visit deeper and deeper levels before visiting more nodes on the parent layers.

For A* Search, a visited list is still used to track which nodes have already been visited, but a disctionary is now used for the frontier list. As a node is discovered, it's heuristic cost is calculated at the time of discovery, and the node:cost pair are added to the dictionary. This allows for both efficient tracking of nodes on the frontier not yet visited and for efficient comparison of cost values to determine the next node to visit.

All of the metrics and results from the algorithm executions are stored as each is ran. At the end of this program, there are two cells to present this: one cell that presents the execution times and number of visited nodes for algorithm performance comparison, and another to provide the list of visited nodes on the frontier for each algoritihm.

### Performance Comparison

In order to compare the performance of the various algorithms, I looked at two key metrics: the time taken for an algorithm to run, and the number of nodes it visited on the frontier while running. These metrics are provided in full for each test case and algorithm in a code cell above the algorithm output. It is worth noting in advance that due to the data size being relatively small, the execution times for these test cases may not be truly representative indicators of performance, and larger variances may be a result of machine performance than algorithm performance.

Upon examining the results, we can see that there is a growing difference as these algorithms are scaled between the three test cases. In the first test case, where `n=25`, there is not a drastic difference in the time taken for each algorithm to compute. In most test runs, absolute best time is achieved by Depth First Search, while the absolute worst time is achieved by A* with Chebyshev Distance Heuristic. However, there is a noticeable difference in the number of nodes that were visited on the frontier; A* algorithms only visited 9-10 nodes, DFS visited 14 nodes, and BFS visited all 25 nodes.

For the second test case with `n=100`, a bit more separation between algorithms is revealed. For the time taken to execute, the distance between performance of algorithms beings to grow, with A* with Chebyshev Distance Heuristic still being the worst on execution time but A* with Manhattan Distance Heuristic now being the best. On the frontier, BFS visits 74 nodes and DFS visits 65 nodes where A* algorithms only visit 27-30 nodes. This may indicate that some A* algorithms are performing better than BFS/DFS, although there is no clear winner among the three A* heuristics at this point.

In the third test case though, where `n=1000`, the separation between the three algorithms and the individual A* heuristics becomes a little more clear. 

- For the time taken to execute, A* with Euclidean Distance Heuristic comes worst with Breadth First Search closely behind. A* Search with Manhattan Distance Heuristic consistently comes in third place. Depth First Search comes in second, and A* with Chebyshev Distance Heuristic constantly comes ahead of other algorithms.

- With the nodes visited on the frontier, a somewhat similar pattern is found. Breadth First Search visited 977 nodes, which is almost the full data set. A* Search with Euclidian Distance Heuristic drops this slightly to 935 nodes visited. Depth First Search visited 741 nodes, and A* Search with Manhattan Distance Heuristic makes a slight jump down to 650 nodes visited. A* Search with Chebyshev Distance Heuristic was the winner here too with only 327 visited nodes.

Together, these two metrics show that, as the data size scales, the algorithms increasingly perform differently. Based on these results, Breadth First Search is likely the worst algorithm for this data set, and __A* Search with Chebyshev Distance Heuristic is likely the best for this data set__. A* algorithms seem more likely to scale with the size of the data if a good heuristic is chosen (for instance, even though the Chebyshev Distance Heuristic seemed to fall behind with small data size on the first test case, it ended up being the fastest and visited the least nodes on the larger third test case). While the three heuristics may perform differently on various data sets, it seems that Chebyshev offers the best performance and scaling for data sets similar to the test cases provided. Reiterating though, the performance of these algorithms may differ with a much larger set, and the trends found here may not indicate large-data performance.



In [224]:
'''
UTILITY CLASS AND DATA PREPARATION

This block will contain utility functions used to prepare data (from files) for the various algorithms.
'''

# Module Imports
import csv
import math
import time
import tabulate

# Creation of utility class to handle data preparation
class DataUtilities:

	# Class Constructor
	def __init__(self, nodeIdFileName, edgeListFileName):
		# Handle data preparation for node ID's
		self.nodeIdFileName = nodeIdFileName
		self.nodeMatrix = []
		self.nodeDict = {}
		self.buildNodeIdMatrix()

		# Handle data preparation for edge list
		self.edgeListFileName = edgeListFileName
		self.edgeDict = {}
		self.buildEdgeAdjacencyDict()

	# Util Function: Get Node IDs from a CSV file, store in matrix and dict
	def buildNodeIdMatrix(self):
		with open(self.nodeIdFileName, 'r') as nodeFile:
			nodeFileReader = csv.reader(nodeFile)
			for row in nodeFileReader:
				nodeId = row[0]
				rowIndex = int(row[1])
				colIndex = int(row[2])

				# If row has not been created yet, add it to matrix
				if rowIndex == len(self.nodeMatrix):
					self.nodeMatrix.append([])

				# Add nodeId at the specified row and column
				self.nodeMatrix[rowIndex].insert(colIndex, nodeId)

				# Add nodeId to dict with its row and column index
				self.nodeDict[nodeId] = (rowIndex, colIndex)

	# Util Function: Get edges with weights from a TXT file and store in dict
	def buildEdgeAdjacencyDict(self):
		with open(self.edgeListFileName, 'r') as edgeFile:
			for line in edgeFile:
				lineSplit = line.split(',')

				# If the edges have not been added to the dict yet, add them
				if lineSplit[0] not in self.edgeDict:
					self.edgeDict[lineSplit[0]] = {}
				if lineSplit[1] not in self.edgeDict:
					self.edgeDict[lineSplit[1]] = {}

				# Add each edge to the adjacency dict
				self.edgeDict[lineSplit[0]][lineSplit[1]] = float(lineSplit[2].strip())
				self.edgeDict[lineSplit[1]][lineSplit[0]] = float(lineSplit[2].strip())

	# Util Function: Get class node ID matrix
	def getNodeIdMatrix(self):
		return self.nodeMatrix
	
	# Util Function: Get class node ID dict
	def getNodeIdDict(self):
		return self.nodeDict

	# Util Function: Get class edge adjacency dict
	def getEdgeAdjacencyDict(self):
		return self.edgeDict

# Creation of timer class for algorithm timing
class AlgorithmTimer:

	# Class Constructor
	def __init__(self):
		self.TimeRecords = {}

	# Timer Function: Start timing for a given test case and algorithm
	def startTimer(self, testCaseName, algorithmName):
		if testCaseName not in self.TimeRecords:
			self.TimeRecords[testCaseName] = {}

		if algorithmName not in self.TimeRecords[testCaseName]:
			self.TimeRecords[testCaseName][algorithmName] = {}

		self.TimeRecords[testCaseName][algorithmName]['start'] = time.time()

	# Timer Function: End timing for a given test case and algorithm then calculate duration in ms
	def endTimer(self, testCaseName, algorithmName):
		self.TimeRecords[testCaseName][algorithmName]['end'] = time.time()
		self.TimeRecords[testCaseName][algorithmName]['duration'] = (self.TimeRecords[testCaseName][algorithmName]['end'] - self.TimeRecords[testCaseName][algorithmName]['start']) * 1000

	# Timer Function: Get the duration of a given test case and algorithm
	def getDuration(self, testCaseName, algorithmName):
		return self.TimeRecords[testCaseName][algorithmName]['duration']
	
# Creation of records class for algorithm results
class AlgorithmRecords:
	
	# Class Constructor
	def __init__(self):
		self.records = {}

	# Record Function: Add a record for a given test case and algorithm
	def addRecord(self, testCaseName, algorithmName, result):
		if testCaseName not in self.records:
			self.records[testCaseName] = {}
		self.records[testCaseName][algorithmName] = result

	# Record Function: Get records for a given test case
	def getRecords(self, testCaseName, algorithmName):
		return self.records[testCaseName][algorithmName]

# Create data utility instances for each test case
TestCase_01_Data = DataUtilities('TestCase_01_NodeID.csv', 'TestCase_01_EdgeList.txt')
TestCase_02_Data = DataUtilities('TestCase_02_NodeID.csv', 'TestCase_02_EdgeList.txt')
TestCase_03_Data = DataUtilities('TestCase_03_NodeID.csv', 'TestCase_03_EdgeList.txt')

# Create timer instance for algorithm timing
Timer = AlgorithmTimer()

# Create records instance for algorithm results
Records = AlgorithmRecords()

In [225]:
'''
BREADTH FIRST SEARCH (BFS) ALGORITHM

This block will implement BFS for each test case and output the list of nodes visited on the frontier.
'''

def performSearchAlgorithm_BFS(nodeIdMatrix, edgeAdjacencyDict):
	# Variable Initialization
	startNode = nodeIdMatrix[0][0]
	endNode = nodeIdMatrix[-1][-1]
	visitedList = []
	frontierList = []

	# Begin frontier with start node
	frontierList.append(startNode)

	# Begin iteration through the frontier
	for currentNode in frontierList:
		# Mark the current node as visited
		visitedList.append(currentNode)

		# If the end node has been reached, break the loop
		if currentNode == endNode:
			break

		# Add the current node's edges to the end of the frontier if it has not been seen yet
		for edgeNode in edgeAdjacencyDict[currentNode].keys():
			if edgeNode not in frontierList and edgeNode not in visitedList:
				frontierList.append(edgeNode)
	
	# Return list of nodes visited on the frontier
	return visitedList

# Perform BFS for Test Case 01 with timing
Timer.startTimer('TestCase_01', 'BFS')
Records.addRecord('TestCase_01', 'BFS', performSearchAlgorithm_BFS(TestCase_01_Data.getNodeIdMatrix(), TestCase_01_Data.getEdgeAdjacencyDict()))
Timer.endTimer('TestCase_01', 'BFS')

# Perform BFS for Test Case 02 with timing
Timer.startTimer('TestCase_02', 'BFS')
Records.addRecord('TestCase_02', 'BFS', performSearchAlgorithm_BFS(TestCase_02_Data.getNodeIdMatrix(), TestCase_02_Data.getEdgeAdjacencyDict()))
Timer.endTimer('TestCase_02', 'BFS')

# Perform BFS for Test Case 03 with timing
Timer.startTimer('TestCase_03', 'BFS')
Records.addRecord('TestCase_03', 'BFS', performSearchAlgorithm_BFS(TestCase_03_Data.getNodeIdMatrix(), TestCase_03_Data.getEdgeAdjacencyDict()))
Timer.endTimer('TestCase_03', 'BFS')
	

In [226]:
'''
DEPTH FIRST SEARCH (DFS) ALGORITHM

This block will implement DFS for each test case and output the list of nodes visited on the frontier.
'''

def performSearchAlgorithm_DFS(nodeIdMatrix, edgeAdjacencyDict):
	# Variable Initialization
	startNode = nodeIdMatrix[0][0]
	endNode = nodeIdMatrix[-1][-1]
	visitedList = []
	frontierList = []

	# Begin frontier with start node
	frontierList.append(startNode)

	# Begin iteration through the frontier
	for currentNode in frontierList:
		# Mark the current node as visited
		visitedList.append(currentNode)

		# If the end node has been reached, break the loop
		if currentNode == endNode:
			break

		# Add the current node's edges to the front of the frontier if it has not been seen yet
		for edgeNode in edgeAdjacencyDict[currentNode].keys():
			if edgeNode not in frontierList and edgeNode not in visitedList:
				frontierList.insert(len(visitedList), edgeNode)
	
	# Return list of nodes visited on the frontier
	return visitedList

# Perform DFS for Test Case 01 with timing
Timer.startTimer('TestCase_01', 'DFS')
Records.addRecord('TestCase_01', 'DFS', performSearchAlgorithm_DFS(TestCase_01_Data.getNodeIdMatrix(), TestCase_01_Data.getEdgeAdjacencyDict()))
Timer.endTimer('TestCase_01', 'DFS')

# Perform DFS for Test Case 02 with timing
Timer.startTimer('TestCase_02', 'DFS')
Records.addRecord('TestCase_02', 'DFS', performSearchAlgorithm_DFS(TestCase_02_Data.getNodeIdMatrix(), TestCase_02_Data.getEdgeAdjacencyDict()))
Timer.endTimer('TestCase_02', 'DFS')

# Perform DFS for Test Case 03 with timing
Timer.startTimer('TestCase_03', 'DFS')
Records.addRecord('TestCase_03', 'DFS', performSearchAlgorithm_DFS(TestCase_03_Data.getNodeIdMatrix(), TestCase_03_Data.getEdgeAdjacencyDict()))
Timer.endTimer('TestCase_03', 'DFS')


In [227]:
'''
A* SEARCH ALGORITHM WITH EUCLIDEAN DISTANCE HEURISTIC

This block will implement A* Search for each test case and output the list of nodes visited on the frontier.
'''

def performSearchAlgorithm_AStar_Euclidean(nodeIdMatrix, nodeIdDict, edgeAdjacencyDict):
	# Variable Initialization
	startNode = nodeIdMatrix[0][0]
	endNode = nodeIdMatrix[-1][-1]
	runningCost = 0
	visitedList = []
	frontierDict = {}

	# Begin frontier with start node
	frontierDict[startNode] = 0

	# Begin iteration through the frontier
	currentNode = startNode
	while endNode not in visitedList:
		# Get the node on the frontier with lowest f cost and mark as visited
		currentNode = min(frontierDict, key=frontierDict.get)
		visitedList.append(currentNode)
		frontierDict.pop(currentNode)

		# If the end node has been reached, break the loop
		if currentNode == endNode:
			break

		# Add all of the child nodes to the frontier if they have not been seen yet
		for edgeNode in edgeAdjacencyDict[currentNode].keys():
			if edgeNode not in frontierDict and edgeNode not in visitedList:
				# Calculate the g cost to reach this node
				gCost = runningCost + edgeAdjacencyDict[currentNode][edgeNode]

				# Estimate the h cost to reach the end node using Euclidean distance
				(rowIndex, colIndex) = nodeIdDict[edgeNode]
				hCost = math.sqrt( pow(((len(nodeIdMatrix[-1])) - (rowIndex)), 2) + pow((len(nodeIdMatrix) - (colIndex)), 2) )

				# Calculate the f cost for total
				fCost = gCost + hCost

				# Add the node to the frontier with its f cost
				frontierDict[edgeNode] = fCost
		
	# Return list of nodes visited on the frontier
	return visitedList

# Perform A* Search with Euclidean distance heuristic for Test Case 01
Timer.startTimer('TestCase_01', 'A* Euclidean')
Records.addRecord('TestCase_01', 'A* Euclidean', performSearchAlgorithm_AStar_Euclidean(TestCase_01_Data.getNodeIdMatrix(), TestCase_01_Data.getNodeIdDict(), TestCase_01_Data.getEdgeAdjacencyDict()))
Timer.endTimer('TestCase_01', 'A* Euclidean')

# Perform A* Search with Euclidean distance heuristic for Test Case 02
Timer.startTimer('TestCase_02', 'A* Euclidean')
Records.addRecord('TestCase_02', 'A* Euclidean', performSearchAlgorithm_AStar_Euclidean(TestCase_02_Data.getNodeIdMatrix(), TestCase_02_Data.getNodeIdDict(), TestCase_02_Data.getEdgeAdjacencyDict()))
Timer.endTimer('TestCase_02', 'A* Euclidean')

# Perform A* Search with Euclidean distance heuristic for Test Case 03
Timer.startTimer('TestCase_03', 'A* Euclidean')
Records.addRecord('TestCase_03', 'A* Euclidean', performSearchAlgorithm_AStar_Euclidean(TestCase_03_Data.getNodeIdMatrix(), TestCase_03_Data.getNodeIdDict(), TestCase_03_Data.getEdgeAdjacencyDict()))
Timer.endTimer('TestCase_03', 'A* Euclidean')

In [228]:
'''
A* SEARCH ALGORITHM WITH MANHATTAN DISTANCE HEURISTIC

This block will implement A* Search for each test case and output the list of nodes visited on the frontier.
'''

def performSearchAlgorithm_AStar_Manhattan(nodeIdMatrix, nodeIdDict, edgeAdjacencyDict):
	# Variable Initialization
	startNode = nodeIdMatrix[0][0]
	endNode = nodeIdMatrix[-1][-1]
	runningCost = 0
	visitedList = []
	frontierDict = {}

	# Begin frontier with start node
	frontierDict[startNode] = 0

	# Begin iteration through the frontier
	currentNode = startNode
	while endNode not in visitedList:
		# Get the node on the frontier with lowest f cost and mark as visited
		currentNode = min(frontierDict, key=frontierDict.get)
		visitedList.append(currentNode)
		frontierDict.pop(currentNode)

		# If the end node has been reached, break the loop
		if currentNode == endNode:
			break

		# Add all of the child nodes to the frontier if they have not been seen yet
		for edgeNode in edgeAdjacencyDict[currentNode].keys():
			if edgeNode not in frontierDict and edgeNode not in visitedList:
				# Calculate the g cost to reach this node
				gCost = runningCost + edgeAdjacencyDict[currentNode][edgeNode]

				# Estimate the h cost to reach the end node using Manhattan distance
				(rowIndex, colIndex) = nodeIdDict[edgeNode]
				hCost = abs((len(nodeIdMatrix) - rowIndex)) + abs((len(nodeIdMatrix[-1]) - colIndex))

				# Calculate the f cost for total
				fCost = gCost + hCost

				# Add the node to the frontier with its f cost
				frontierDict[edgeNode] = fCost
		
	# Return list of nodes visited on the frontier
	return visitedList

# Perform A* Search with Manhattan distance heuristic for Test Case 01
Timer.startTimer('TestCase_01', 'A* Manhattan')
Records.addRecord('TestCase_01', 'A* Manhattan', performSearchAlgorithm_AStar_Manhattan(TestCase_01_Data.getNodeIdMatrix(), TestCase_01_Data.getNodeIdDict(), TestCase_01_Data.getEdgeAdjacencyDict()))
Timer.endTimer('TestCase_01', 'A* Manhattan')

# Perform A* Search with Manhattan distance heuristic for Test Case 02
Timer.startTimer('TestCase_02', 'A* Manhattan')
Records.addRecord('TestCase_02', 'A* Manhattan', performSearchAlgorithm_AStar_Manhattan(TestCase_02_Data.getNodeIdMatrix(), TestCase_02_Data.getNodeIdDict(), TestCase_02_Data.getEdgeAdjacencyDict()))
Timer.endTimer('TestCase_02', 'A* Manhattan')

# Perform A* Search with Manhattan distance heuristic for Test Case 03
Timer.startTimer('TestCase_03', 'A* Manhattan')
Records.addRecord('TestCase_03', 'A* Manhattan', performSearchAlgorithm_AStar_Manhattan(TestCase_03_Data.getNodeIdMatrix(), TestCase_03_Data.getNodeIdDict(), TestCase_03_Data.getEdgeAdjacencyDict()))
Timer.endTimer('TestCase_03', 'A* Manhattan')

In [229]:
'''
A* SEARCH ALGORITHM WITH CHEBYSHEV DISTANCE HEURISTIC

This block will implement A* Search for each test case and output the list of nodes visited on the frontier.
'''

def performSearchAlgorithm_AStar_Chebyshev(nodeIdMatrix, nodeIdDict, edgeAdjacencyDict):
	# Variable Initialization
	startNode = nodeIdMatrix[0][0]
	endNode = nodeIdMatrix[-1][-1]
	runningCost = 0
	visitedList = []
	frontierDict = {}

	# Begin frontier with start node
	frontierDict[startNode] = 0

	# Begin iteration through the frontier
	currentNode = startNode
	while endNode not in visitedList:
		# Get the node on the frontier with lowest f cost and mark as visited
		currentNode = min(frontierDict, key=frontierDict.get)
		visitedList.append(currentNode)
		frontierDict.pop(currentNode)

		# If the end node has been reached, break the loop
		if currentNode == endNode:
			break

		# Add all of the child nodes to the frontier if they have not been seen yet
		for edgeNode in edgeAdjacencyDict[currentNode].keys():
			if edgeNode not in frontierDict and edgeNode not in visitedList:
				# Calculate the g cost to reach this node
				gCost = runningCost + edgeAdjacencyDict[currentNode][edgeNode]

				# Estimate the h cost to reach the end node using Chebyshev distance
				(rowIndex, colIndex) = nodeIdDict[edgeNode]
				hCost = max(abs((len(nodeIdMatrix) - rowIndex)), abs((len(nodeIdMatrix[-1]) - colIndex)))

				# Calculate the f cost for total
				fCost = gCost + hCost

				# Add the node to the frontier with its f cost
				frontierDict[edgeNode] = fCost
		
	# Return list of nodes visited on the frontier
	return visitedList

# Perform A* Search with Chebyshev distance heuristic for Test Case 01
Timer.startTimer('TestCase_01', 'A* Chebyshev')
Records.addRecord('TestCase_01', 'A* Chebyshev', performSearchAlgorithm_AStar_Chebyshev(TestCase_01_Data.getNodeIdMatrix(), TestCase_01_Data.getNodeIdDict(), TestCase_01_Data.getEdgeAdjacencyDict()))
Timer.endTimer('TestCase_01', 'A* Chebyshev')

# Perform A* Search with Chebyshev distance heuristic for Test Case 02
Timer.startTimer('TestCase_02', 'A* Chebyshev')
Records.addRecord('TestCase_02', 'A* Chebyshev', performSearchAlgorithm_AStar_Chebyshev(TestCase_02_Data.getNodeIdMatrix(), TestCase_02_Data.getNodeIdDict(), TestCase_02_Data.getEdgeAdjacencyDict()))
Timer.endTimer('TestCase_02', 'A* Chebyshev')

# Perform A* Search with Chebyshev distance heuristic for Test Case 03
Timer.startTimer('TestCase_03', 'A* Chebyshev')
Records.addRecord('TestCase_03', 'A* Chebyshev', performSearchAlgorithm_AStar_Chebyshev(TestCase_03_Data.getNodeIdMatrix(), TestCase_03_Data.getNodeIdDict(), TestCase_03_Data.getEdgeAdjacencyDict()))
Timer.endTimer('TestCase_03', 'A* Chebyshev')

In [230]:
'''
ALGORITHM ANALYSIS

This block will print metrics to compare algorithms on each test case.
'''

### Test Case 01 Analysis ###

print(f"Algorithm Metrics for Test Case 01")
TestCase_01_Results_Headers = ["Algorithm Name", "Number Nodes Visited on Frontier", "Time Taken (ms)"]
TestCase_01_Results_Table = [["Breadth First Search (BFS)", len(Records.getRecords('TestCase_01', 'BFS')), Timer.getDuration("TestCase_01", "BFS")],
							["Depth First Search (DFS)", len(Records.getRecords('TestCase_01', 'DFS')), Timer.getDuration("TestCase_01", "DFS")],
							["A* Search with Euclidean Distance Heuristic", len(Records.getRecords('TestCase_01', 'A* Euclidean')), Timer.getDuration("TestCase_01", "A* Euclidean")],
							["A* Search with Manhattan Distance Heuristic", len(Records.getRecords('TestCase_01', 'A* Manhattan')), Timer.getDuration("TestCase_01", "A* Manhattan")],
							["A* Search with Chebyshev Distance Heuristic", len(Records.getRecords('TestCase_01', 'A* Chebyshev')), Timer.getDuration("TestCase_01", "A* Chebyshev")]]
print(tabulate.tabulate(TestCase_01_Results_Table, TestCase_01_Results_Headers, tablefmt="github"))

### Test Case 02 Analysis ###

print(f"\n\nAlgorithm Metrics for Test Case 02")
TestCase_02_Results_Headers = ["Algorithm Name", "Number Nodes Visited on Frontier", "Time Taken (ms)"]
TestCase_02_Results_Table = [["Breadth First Search (BFS)", len(Records.getRecords('TestCase_02', 'BFS')), Timer.getDuration("TestCase_02", "BFS")],
							["Depth First Search (DFS)", len(Records.getRecords('TestCase_02', 'DFS')), Timer.getDuration("TestCase_02", "DFS")],
							["A* Search with Euclidean Distance Heuristic", len(Records.getRecords('TestCase_02', 'A* Euclidean')), Timer.getDuration("TestCase_02", "A* Euclidean")],
							["A* Search with Manhattan Distance Heuristic", len(Records.getRecords('TestCase_02', 'A* Manhattan')), Timer.getDuration("TestCase_02", "A* Manhattan")],
							["A* Search with Chebyshev Distance Heuristic", len(Records.getRecords('TestCase_02', 'A* Chebyshev')), Timer.getDuration("TestCase_02", "A* Chebyshev")]]
print(tabulate.tabulate(TestCase_02_Results_Table, TestCase_02_Results_Headers, tablefmt="github"))

### Test Case 03 Analysis ###

print(f"\n\nAlgorithm Metrics for Test Case 03")
TestCase_03_Results_Headers = ["Algorithm Name", "Number Nodes Visited on Frontier", "Time Taken (ms)"]
TestCase_03_Results_Table = [["Breadth First Search (BFS)", len(Records.getRecords('TestCase_03', 'BFS')), Timer.getDuration("TestCase_03", "BFS")],
							["Depth First Search (DFS)", len(Records.getRecords('TestCase_03', 'DFS')), Timer.getDuration("TestCase_03", "DFS")],
							["A* Search with Euclidean Distance Heuristic", len(Records.getRecords('TestCase_03', 'A* Euclidean')), Timer.getDuration("TestCase_03", "A* Euclidean")],
							["A* Search with Manhattan Distance Heuristic", len(Records.getRecords('TestCase_03', 'A* Manhattan')), Timer.getDuration("TestCase_03", "A* Manhattan")],
							["A* Search with Chebyshev Distance Heuristic", len(Records.getRecords('TestCase_03', 'A* Chebyshev')), Timer.getDuration("TestCase_03", "A* Chebyshev")]]
print(tabulate.tabulate(TestCase_03_Results_Table, TestCase_03_Results_Headers, tablefmt="github"))

Algorithm Metrics for Test Case 01
| Algorithm Name                              |   Number Nodes Visited on Frontier |   Time Taken (ms) |
|---------------------------------------------|------------------------------------|-------------------|
| Breadth First Search (BFS)                  |                                 25 |         0.0772476 |
| Depth First Search (DFS)                    |                                 14 |         0.0457764 |
| A* Search with Euclidean Distance Heuristic |                                  9 |         0.079155  |
| A* Search with Manhattan Distance Heuristic |                                 10 |         0.0488758 |
| A* Search with Chebyshev Distance Heuristic |                                  9 |         0.0510216 |


Algorithm Metrics for Test Case 02
| Algorithm Name                              |   Number Nodes Visited on Frontier |   Time Taken (ms) |
|---------------------------------------------|------------------------------------|----

In [231]:
'''
ALGORITHM OUTPUT

This block will print the results for each algorithm on each test case.
'''

### Test Case 01 Output ###

print(f"Algorithm Outputs for Test Case 01:")
print(f"| Breadth First Search (BFS) Frontier: {Records.getRecords('TestCase_01', 'BFS')}")
print(f"| Depth First Search (DFS) Frontier: {Records.getRecords('TestCase_01', 'DFS')}")
print(f"| A* Search with Euclidean Distance Heuristic Frontier: {Records.getRecords('TestCase_01', 'A* Euclidean')}")
print(f"| A* Search with Manhattan Distance Heuristic Frontier: {Records.getRecords('TestCase_01', 'A* Manhattan')}")
print(f"| A* Search with Chebyshev Distance Heuristic Frontier: {Records.getRecords('TestCase_01', 'A* Chebyshev')}")

### Test Case 02 Output ###

print(f"\nAlgorithm Outputs for Test Case 02:")
print(f"| Breadth First Search (BFS) Frontier: {Records.getRecords('TestCase_02', 'BFS')}")
print(f"| Depth First Search (DFS) Frontier: {Records.getRecords('TestCase_02', 'DFS')}")
print(f"| A* Search with Euclidean Distance Heuristic Frontier: {Records.getRecords('TestCase_02', 'A* Euclidean')}")
print(f"| A* Search with Manhattan Distance Heuristic Frontier: {Records.getRecords('TestCase_02', 'A* Manhattan')}")
print(f"| A* Search with Chebyshev Distance Heuristic Frontier: {Records.getRecords('TestCase_02', 'A* Chebyshev')}")

### Test Case 03 Output ###

print(f"\nAlgorithm Outputs for Test Case 03:")
print(f"| Breadth First Search (BFS) Frontier: {Records.getRecords('TestCase_03', 'BFS')}")
print(f"| Depth First Search (DFS) Frontier: {Records.getRecords('TestCase_03', 'DFS')}")
print(f"| A* Search with Euclidean Distance Heuristic Frontier: {Records.getRecords('TestCase_03', 'A* Euclidean')}")
print(f"| A* Search with Manhattan Distance Heuristic Frontier: {Records.getRecords('TestCase_03', 'A* Manhattan')}")
print(f"| A* Search with Chebyshev Distance Heuristic Frontier: {Records.getRecords('TestCase_03', 'A* Chebyshev')}")

Algorithm Outputs for Test Case 01:
| Breadth First Search (BFS) Frontier: ['N_0', 'N_1', 'N_6', 'N_2', 'N_5', 'N_7', 'N_3', 'N_10', 'N_12', 'N_11', 'N_15', 'N_13', 'N_17', 'N_16', 'N_20', 'N_14', 'N_8', 'N_18', 'N_22', 'N_21', 'N_9', 'N_19', 'N_23', 'N_4', 'N_24']
| Depth First Search (DFS) Frontier: ['N_0', 'N_1', 'N_2', 'N_3', 'N_6', 'N_7', 'N_12', 'N_17', 'N_22', 'N_23', 'N_13', 'N_18', 'N_19', 'N_24']
| A* Search with Euclidean Distance Heuristic Frontier: ['N_0', 'N_1', 'N_6', 'N_7', 'N_12', 'N_13', 'N_18', 'N_19', 'N_24']
| A* Search with Manhattan Distance Heuristic Frontier: ['N_0', 'N_1', 'N_6', 'N_7', 'N_12', 'N_13', 'N_14', 'N_18', 'N_19', 'N_24']
| A* Search with Chebyshev Distance Heuristic Frontier: ['N_0', 'N_1', 'N_6', 'N_7', 'N_12', 'N_13', 'N_18', 'N_19', 'N_24']

Algorithm Outputs for Test Case 02:
| Breadth First Search (BFS) Frontier: ['N_0', 'N_10', 'N_20', 'N_11', 'N_1', 'N_2', 'N_3', 'N_12', 'N_4', 'N_13', 'N_14', 'N_23', 'N_33', 'N_24', 'N_22', 'N_34', 'N_25',